# Setting up Codespaces

1. In the bottom terminal, let Codespaces configure itself and press any key to exit.  
2. On the top right, click **Select Kernel** and install/enable the suggested extensions: **Python** + **Jupyter**.  
3. Click **Select Kernel → Python Environments → Python 3.12.1**.  
4. You are now ready to run the cells below!

# Install pyzentropy in editable mode

In [ ]:
!cd ../../ && pip install -e .

# Load modules

In [ ]:
# Enable autoreload of modules
%reload_ext autoreload
%autoreload 2

# Third-party imports
import numpy as np
import pickle

# PyZentropy imports
from pyzentropy.configuration import Configuration
from pyzentropy.system import System

# Load Fe₃Pt Data

This section imports the **three lowest-energy configurations** of the 12-atom Fe₃Pt supercell:  
- **FM (Ferromagnetic):** non-zero total magnetic moment  
- **SF28 (Spin-Flipping):** specific spin-flip arrangement (#28) with unequal spin-up and spin-down populations  
- **SF22 (Spin-Flipping):** specific spin-flip arrangement (#22) with unequal spin-up and spin-down populations  

The pickle file contains Helmholtz free energies, their derivatives, entropies, and heat capacities, all precomputed using the **DFTTK** package.

In [ ]:
# Load the Fe3Pt input data
with open("Fe3Pt_input.pkl", "rb") as f:
    input = pickle.load(f)

# PyZentropy Configuration Objects

This section constructs PyZentropy `Configuration` objects for each Fe₃Pt configuration. For each configuration, the internal energies are calculated automatically.

In [ ]:
# Initialize dictionary to hold Configuration objects
config_objects = {}

for config_name in input:
    config = input[config_name]

    # Create a Configuration object for each config and store in the dictionary
    config_objects[config_name] = Configuration(
        name=config_name,
        multiplicity=config['multiplicity'],
        number_of_atoms=config['number_of_atoms'],
        volumes=config['volumes'],
        temperatures=config['temperatures'],
        # Extract Helmholtz energies and their derivatives from DFTTK results
        helmholtz_energies=config['helmholtz_energies'],
        helmholtz_energies_dV=config['helmholtz_energies_dV'],
        helmholtz_energies_d2V2=config['helmholtz_energies_d2V2'],
        # Extract entropy and heat capacity data
        entropies=config['entropies'],
        heat_capacities=config['heat_capacities']
    )

Plot Helmholtz energy as a function of volume for a single configuration.

You can change the configuration name (e.g., "FM") to plot other configurations.

**Available plot types:**
- "helmholtz_energy_vs_volume"
- "helmholtz_energy_vs_temperature"
- "internal_energy_vs_volume"
- "internal_energy_vs_temperature"
- "entropy_vs_volume"
- "entropy_vs_temperature"
- "heat_capacity_vs_volume"
- "heat_capacity_vs_temperature"


In [ ]:
config_objects["FM"].plot_vt("helmholtz_energy_vs_volume")

In [ ]:
# You can also plot for selected temperatures by passing an array of temperatures.
# The code will automatically find and use the closest available temperatures.
selected_temperatures = np.array([100, 300, 600, 900])
config_objects["FM"].plot_vt("helmholtz_energy_vs_volume", selected_temperatures=selected_temperatures)

In [ ]:
# You can also plot for selected volumes by passing an array of volumes.
# The code will automatically find and use the closest available volumes.
selected_volumes = np.array([136, 140, 160])
config_objects["FM"].plot_vt("helmholtz_energy_vs_temperature", selected_volumes=selected_volumes)

# PyZentropy System Objects

This section constructs a PyZentropy `System` object using all Fe₃Pt configurations. Using the zentropy equations, the system partition function, Helmholtz energy and its derivatives, entropy, bulk modulus, heat capacities, and phase diagrams are calculated. Configuration probabilities are also determined.

In [ ]:
# Create a System object from the configuration objects
system = System(config_objects, ground_state="FM")

In [ ]:
# Calculate phase diagrams with specified ground state configuration. 
# This also calculates all the properties at constant pressure.
system.calculate_phase_diagrams()

The following are example plots vs. volume or temperature. For all of these plots (apart from vt_phase_diagram), you can plot for `selected_temperatures` and `selected_volumes`. The available plots are:

**Helmholtz Energy**
- "helmholtz_energy_vs_volume"
- "helmholtz_energy_vs_temperature"
- "helmholtz_energy_dV_vs_volume"
- "helmholtz_energy_dV_vs_temperature"
- "helmholtz_energy_d2V2_vs_volume"
- "helmholtz_energy_d2V2_vs_temperature"

**Entropy**
- "entropy_vs_volume"
- "entropy_vs_temperature"
- "configurational_entropy_vs_volume"
- "configurational_entropy_vs_temperature"

**Heat Capacity**
- "heat_capacity_vs_volume"
- "heat_capacity_vs_temperature"

**Bulk Modulus**
- "bulk_modulus_vs_volume"
- "bulk_modulus_vs_temperature"

**Phase Diagram**
- "vt_phase_diagram"


In [ ]:
system.plot_vt("helmholtz_energy_vs_volume")

In [ ]:
system.plot_vt("vt_phase_diagram")

The following are example plots for properties at constant pressure. You can plot for `selected_temperatures` for helmholtz_energy_pv_vs_volume. The available plots are:

- "probability_vs_temperature"
- "helmholtz_energy_pv_vs_volume"
- "volume_vs_temperature"
- "CTE_vs_temperature"
- "LCTE_vs_temperature"
- "entropy_vs_temperature"
- "configurational_entropy_vs_temperature"
- "heat_capacity_vs_temperature"
- "gibbs_energy_vs_temperature"
- "bulk_modulus_vs_temperature"
- "pt_phase_diagram"


In [ ]:
system.plot_pt("pt_phase_diagram")

In [ ]:
system.plot_pt("probability_vs_temperature", P=0)

In [ ]:
system.plot_pt("volume_vs_temperature", P=0)

In [ ]:
system.plot_pt("LCTE_vs_temperature", P=0)

In [ ]:
system.plot_pt("entropy_vs_temperature", P=0)

In [ ]:
system.plot_pt("heat_capacity_vs_temperature", P=0)